# 「対話システムの評価・調査」用実験プログラム

このプログラムは、「対話システムの評価・調査」における実験を行うためのものです。実験前調査に回答し終わっている場合は、実験の注意事項をよく読んだ上で、以下の手順に従って実験を行ってください。
  
<br>

#実験手順
行程1, 行程2, 行程3, 行程4の順番で実験を進めてください。途中で実験の方法や手順が分からなくなったときは、実験者に連絡してください。メールアドレス：b1018134@fun.ac.jp

<br>

#注意事項
* いかなる場合においても、このプログラムを変更、複写、保存、その他実験に使用する以外の用途では使用しないでください。

* 対話は、システムと行っているということを意識せず、人間と行っていると考えて行ってください。システムは、自分が「日常生活を送る普通の人間」である前提で話します。

* システムは、現時点で固有名詞や人名の処理が得意ではありません。また、入力文が文法的に間違っていたり、スラングや短縮語が多く含まれていたり、文が短すぎるとうまく返答できない可能性があります。これらについて、できるだけ注意して対話を行ってみてください。

* システムとの対話が不可能なレベルで応答文が破綻している場合、もしくは同じ応答が繰り返されるようになった場合はこちらから話を転換してみてください。

* 実験にて入力した文章は、ログに保存されます。また、保存したログは、調査フォームにて提出していただきます。これらのデータは、調査フォームの「回答に置ける注意事項」に記載されたデータの取り扱いに沿って管理、処理します。

* 特定の人物や団体、その他差別や誹謗中傷につながるような不適切な表現が応答として出力された場合、直ちに実験を終了し、実験者に報告してください。

* その他、実験の続行が不可能であると判断した場合も実験を中断してかまいません。

* 上部メニューバーの「ランタイム」より、「ランタイムのタイプを変更」でランタイムのタイプを「None」から「GPU」にすると、準備部分や実験中の処理が速くなります。



# 準備部分
# **行程1.実験開始前に、このセルの実行ボタン(「20個のセルが非表示」の左部のボタン)を<br>押下してください。(これには数分かかる場合があります。)**
####*実行ボタンの回転が止まったら実行終了です。行程2にお進みください。

## 依存ライブラリのインストール

In [ ]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 #torch,torchtext,torchvisionをインストール
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece #transformers,pytorch_lightningをインストール

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
from tqdm import tqdm
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

## ディレクトリ作成

* model: 学習済みモデル格納用

In [ ]:
!mkdir -p /content/premodel /content/model /content/data /content/data/log #ディレクトリを作成

# 転移学習済みモデルのディレクトリ
MODEL_DIR = "/content/model"

In [ ]:
import gdown
filename="5ep10050.zip"
gdown.download('https://drive.google.com/uc?id=14te8HG9U4hvhnR5lUJdv_Fz6Om_ZgINq', filename, quiet=False)

! unzip -u 5ep10050.zip -d /content/premodel 

Downloading...
From: https://drive.google.com/uc?id=14te8HG9U4hvhnR5lUJdv_Fz6Om_ZgINq
To: /content/5ep10050.zip
100%|██████████| 823M/823M [00:06<00:00, 124MB/s] 


Archive:  5ep10050.zip


## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [ ]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s): #unicode正規化
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 情報抽出クラスの定義

In [ ]:
import tarfile
import re 

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{utter}\t{resp}"です。

In [ ]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, utter, resp):#gener_idを削除
        # 用の入出力形式に変換する。
        ainput = f"{utter}"
        target = f"{resp}"
        return ainput, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                utter = line[0]
                resp = line[1]

                ainput, target = self._make_record(utter, resp)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [ainput], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


## 学習済みモデルの読み込み

In [ ]:
DRIVE_DIR = '/content/premodel/models' #マウントしたGoogleDriveのディレクトリ
USE_MODEL=DRIVE_DIR   #学習したモデルを利用するか、ドライブのモデルを利用するか

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(USE_MODEL, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(USE_MODEL)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

## 追加学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 事前学習済みモデルの名称
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    # model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=1e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=64,
    # max_target_length=512,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=10,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


In [ ]:
class T5ReFineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained('/content/premodel/models')

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    # def validation_step(self, batch, batch_idx):
    #     """バリデーションステップ処理"""
    #     loss = self._step(batch)
    #     self.log("val_loss", loss)
    #     return {"val_loss": loss}

    # def test_step(self, batch, batch_idx):
    #     """テストステップ処理"""
    #     loss = self._step(batch)
    #     self.log("test_loss", loss)
    #     return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="add.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            # val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
            #                                type_path="dev.tsv", args=self.hparams)
            # self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    # def val_dataloader(self):
    #     """バリデーションデータローダーを作成する"""
    #     return DataLoader(self.val_dataset, 
    #                       batch_size=self.hparams.eval_batch_size, 
    #                       num_workers=4)


In [ ]:
def make_tsv(add_data):
  random.seed(1234)
  random.shuffle(add_data)

  def to_line(data):
      utter = data["utter"]
      resp = data["resp"]
      #gener_idは不要のため削除

      assert len(utter) > 0 and len(resp) > 0
      return f"{utter}\t{resp}\n"    #gener_idは不要のため削除

  data_size = len(add_data)

  with open(f"data/add.tsv", "w", encoding="utf-8") as f_train:
      
      for i, data in enumerate(add_data):
          line = to_line(data)
          f_train.write(line)

In [ ]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  100,  # 入力文の最大トークン数
    "max_target_length": 50,  # 出力文の最大トークン数
    "train_batch_size":  1,   # 学習バッチサイズ
    "eval_batch_size":   1,   # 推論バッチサイズ
    "num_train_epochs":  5,  # 学習epoch数
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

#**行程2.実験の注意事項を理解していることを確認して、前半の実験を行ってください。**

In [ ]:
#@title 実験前半：左部の実行ボタンを押下して開始します。

MAX_SOURCE_LENGTH = 100   # 入力最大トークン数
MAX_TARGET_LENGTH = 50  # 生成最大トークン数

dotrain=True

def preprocess_utter(text):
    return normalize_text(text.replace("\n", ""))

template=[\
          "早速ですが、好きなものはありますか？",\
          "今日はいい天気ですね。",\
          "スポーツは好きですか？",\
          "ボランティアってやったことありますか？",\
          "子供のころの夢ってなんでしたか？",\
          "SNSってやってますか？",\
          "部屋の片づけをしたのに、いつの間にかまた汚くなってるんですよ。",\
          "昨日一晩中ゲームしちゃって、寝不足です。",\
          "先日、英語で話しかけられたんですけど、答えられなくて。",\
          "AIについてどう思いますか？"\
]
random.shuffle(template)
f_template=template[0:5]
l_template=template[5:10]
# print(f_template,l_template)

#実験手順の説明-----------------------------------------
wait_time = 6 #発話待ち時間

print("sys:これより前半の実験を開始します。")
time.sleep(wait_time)
print("sys:この実験では、あなたに対話を行ってもらいます。")
time.sleep(wait_time)
print("sys:前半では、対話を5セット行っていただきます。")
time.sleep(wait_time)
print("sys:1セットの中では、あなたの入力とシステムの応答が5回繰り返されます。")
time.sleep(wait_time)
print("sys:各セットでは、最初にシステムから話題が提示されますので、できるだけその話題に関する対話を行ってください。")
time.sleep(wait_time)
print("sys:それでは、以上の説明を十分に理解されましたら、「y」と入力し、次へお進みください。")

while(1):
  print("y/n?")
  if(input()=="y"):
    break
  
#-------------------------------------------------------
add_data=[]
log="実験前半のログ\n"
rep_num = 5 #セット繰り返し回数
set_num = 5 #各セットの発話数

for rep in range(rep_num):
  pre_utter = add_resp = f_template[rep] #システムの最初の発話
  pre_log=pre_utter+"\n"
  log+=pre_log
  print(f"sys:これより{rep+1}セット目の対話を開始します。")
  print("sys:「"+pre_utter+"」")
  for set in range(set_num):
    input_utter = input()
    # 推論モード設定
    trained_model.eval()

    # add_data.append([add_resp,input_utter])
    if len(add_resp) > 0 and len(input_utter) > 0:
      add_data.append({
                  "utter": add_resp,
                  "resp": input_utter,
                  })

    # print(add_data)
    
    utter=pre_utter + '<sep>' + input_utter

    pre_utter = input_utter

    # 前処理とトークナイズを行う
    inputs = [preprocess_utter(utter)]
    batch = tokenizer.batch_encode_plus(
        inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
        padding="longest", return_tensors="pt")

    input_ids = batch['input_ids']
    input_mask = batch['attention_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=input_ids, attention_mask=input_mask, 
        max_length=MAX_TARGET_LENGTH,
        temperature=2.0,  # 生成にランダム性を入れる温度パラメータ
        #num_beams=1,  # ビームサーチの探索幅
        #num_return_sequences=1,  # 生成する文の数
        #diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        repetition_penalty=15.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
    )

    # print(trained_model.loss)

    # 生成されたトークン列を文字列に変換する
    generated_bodies = [tokenizer.decode(ids, skip_special_tokens=True, 
                                        clean_up_tokenization_spaces=False) 
                        for ids in outputs]

    # 生成された文章を表示する
    for i, resp in enumerate(generated_bodies):
        print("sys:「"+resp+"」")
        add_resp=resp
    

    logstep=input_utter+","+add_resp+"\n"
    log+=logstep

print("\n")
print("sys:お疲れさまでした。以上で前半の実験を終了します。")
time.sleep(2)
print("sys:一度調査フォームに戻り，前半終了後の調査にお進みください。なお、この実験フォームはすべての実験が終了するまで閉じないでください。")


sys:これより前半の実験を開始します。
sys:この実験では、あなたに対話を行ってもらいます。
sys:前半では、対話を5セット行っていただきます。
sys:1セットの中では、あなたの入力とシステムの応答が5回繰り返されます。
sys:各セットでは、最初にシステムから話題が提示されますので、できるだけその話題に関する対話を行ってください。
sys:それでは、以上の説明を十分に理解されましたら、「y」と入力し、次へお進みください。
y/n?
y
sys:これより1セット目の対話を開始します。
sys:「昨日一晩中ゲームしちゃって、寝不足です。」
あ
sys:「寝るのが怖くて眠れませんでした。」


KeyboardInterrupt: ignored

#**行程3.後半の実験を行う前に、このセルの実行ボタンを押下してください。(これには数分かかる場合があります。)**
####*実行ボタンの回転が止まったら実行終了です。行程4にお進みください。

In [ ]:
if(dotrain):
  make_tsv(add_data)

      # 転移学習の実行
  MODEL_DIR = '/content/model'

  model = T5ReFineTuner(args)
  trainer = pl.Trainer(**train_params)
  trainer.fit(model)

  # 最終エポックのモデルを保存
  model.tokenizer.save_pretrained(MODEL_DIR)
  model.model.save_pretrained(MODEL_DIR)

  del model


Downloading:   0%|          | 0.00/804k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
DRIVE_DIR = '/content/model' #マウントしたGoogleDriveのディレクトリ
USE_MODEL=DRIVE_DIR   #学習したモデルを利用するか、ドライブのモデルを利用するか

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(USE_MODEL, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(USE_MODEL)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

In [ ]:
# DRIVE_DIR = '/content/model' #マウントしたGoogleDriveのディレクトリ
# USE_MODEL=DRIVE_DIR   #学習したモデルを利用するか、ドライブのモデルを利用するか

# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# # トークナイザー（SentencePiece）
# tokenizer = T5Tokenizer.from_pretrained(USE_MODEL, is_fast=True)

# # 学習済みモデル
# trained_model = T5ForConditionalGeneration.from_pretrained(USE_MODEL)

# # GPUの利用有無
# USE_GPU = torch.cuda.is_available()
# if USE_GPU:
#     trained_model.cuda()

#**行程4.後半の実験を行ってください。**

In [ ]:
#@title 実験後半：左部の実行ボタンを押下して開始します。

#実験手順の説明-----------------------------------------
wait_time = 6 #発話待ち時間

print("sys:これより後半の実験を開始します。")
time.sleep(wait_time)
print("sys:後半では、前半と同じくシステムと5セット対話を行っていただきます。")
time.sleep(wait_time)
print("sys:提示される話題は、前半とは異なります。")
time.sleep(wait_time)
print("sys:また、後半においても、前半と同様の注意事項に則ってください。")
time.sleep(wait_time)
print("sys:それでは、以上の説明を十分に理解されましたら、「y」と入力し、次へお進みください。")

while(1):
  print("y/n?")
  if(input()=="y"):
    break
  
#-------------------------------------------------------
rep_num = 5 #セット繰り返し回数
set_num = 5 #各セットの発話数

log+="\n実験後半のログ\n"

for rep in range(rep_num):
  pre_utter = add_resp = l_template[rep] #システムの最初の発話
  pre_log=pre_utter+"\n"
  log+=pre_log
  print(f"sys:これより{rep+1}セット目の対話を開始します。")
  print("sys:「"+pre_utter+"」")
  for set in range(set_num):
    input_utter = input()
    # 推論モード設定
    trained_model.eval()
    
    utter=pre_utter + '<sep>' + input_utter

    pre_utter = input_utter

    # 前処理とトークナイズを行う
    inputs = [preprocess_utter(utter)]
    batch = tokenizer.batch_encode_plus(
        inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
        padding="longest", return_tensors="pt")

    input_ids = batch['input_ids']
    input_mask = batch['attention_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=input_ids, attention_mask=input_mask, 
        max_length=MAX_TARGET_LENGTH,
        temperature=2.0,  # 生成にランダム性を入れる温度パラメータ
        #num_beams=1,  # ビームサーチの探索幅
        #num_return_sequences=1,  # 生成する文の数
        #diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        repetition_penalty=15.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
    )

    # print(trained_model.loss)

    # 生成されたトークン列を文字列に変換する
    generated_bodies = [tokenizer.decode(ids, skip_special_tokens=True, 
                                        clean_up_tokenization_spaces=False) 
                        for ids in outputs]

    # 生成された文章を表示する
    for i, resp in enumerate(generated_bodies):
        print("sys:「"+resp+"」")
        add_resp=resp
    logstep=input_utter+","+add_resp+"\n"
    log+=logstep

f = open('log.txt', 'w')
f.write(log)
f.close()

print("\n")
print("sys:お疲れさまでした。以上で実験はすべて終了となります。")
time.sleep(wait_time)
print("sys:本実験サイトの左部にあるメニューの「ファイル」から「log.txt」を右クリックしてダウンロードし、調査フォームからアップロードしてください。")
time.sleep(wait_time)
print("sys:ログファイルの提出ができ次第、実験終了後の調査にお進みください。この実験サイトは閉じても構いません。")


ライセンス

[CC-BY SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/deed.ja)

このプログラム及び仕様モデルは、sonoisaによる「日本語T5事前学習済みモデル 」およびその転移学習のサンプルコードを改変・転移学習したプログラムです。